<a href="https://colab.research.google.com/github/Eldabs/Sistema_Gerenciamento_de_Biblioteca/blob/main/Sistema_Gerenciamento_de_Biblioteca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importa o módulo para trabalhar com banco de dados SQLite
import sqlite3
# Importa classes para manipulação de datas
from datetime import datetime, timedelta

class Biblioteca:
    def __init__(self):
        # Conecta ao banco de dados (cria se não existir)
        self.conn = sqlite3.connect('biblioteca.db')
        # Chama método para criar as tabelas necessárias
        self.criar_tabelas()

    def criar_tabelas(self):
        # Cria todas as tabelas necessárias no banco de dados caso não existam
        # Cria um cursor para executar comandos SQL
        cursor = self.conn.cursor()

        # Cria tabela de livros
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS livros (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titulo TEXT NOT NULL,
            autor TEXT NOT NULL,
            ano_publicacao INTEGER,
            copias_disponiveis INTEGER DEFAULT 0,
            copias_totais INTEGER DEFAULT 0
        )
        ''')

        # Cria tabela de usuários
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS usuarios (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            identificacao TEXT UNIQUE NOT NULL,
            contato TEXT
        )
        ''')

        # Cria tabela de empréstimos
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS emprestimos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            livro_id INTEGER NOT NULL,
            usuario_id INTEGER NOT NULL,
            data_emprestimo TEXT NOT NULL,
            data_devolucao_prevista TEXT NOT NULL,
            data_devolucao_real TEXT,
            FOREIGN KEY (livro_id) REFERENCES livros (id),
            FOREIGN KEY (usuario_id) REFERENCES usuarios (id)
        )
        ''')

        # Confirma as alterações no banco de dados
        self.conn.commit()

    def cadastrar_usuario(self):
        # Interface para cadastro de novos usuários no sistema
        print("\n")
        print("-" * 40)
        print("          Cadastro de Usuário     ")
        print("-" * 40)

        # Loop de validação dos dados
        while True:
            # Solicita e valida nome
            nome = input("Nome completo: ").strip()
            if not nome:
                print("Erro: O nome é obrigatório!")
                continue

            # Solicita e valida identificação
            identificacao = input("Número de identificação: ").strip()
            if not identificacao:
                print("Erro: O número de identificação é obrigatório!")
                continue

            # Solicita e valida contato
            contato = input("Contato (telefone/email): ").strip()
            if not contato:
                print("Erro: O contato é obrigatório!")
                continue

            break  # Sai do loop quando todos os dados são válidos

        try:
            # Prepara o cursor para execução SQL
            cursor = self.conn.cursor()
            # Insere os dados do novo usuário
            cursor.execute('''
            INSERT INTO usuarios (nome, identificacao, contato)
            VALUES (?, ?, ?)
            ''', (nome, identificacao, contato))

            # Obtém o ID gerado automaticamente
            usuario_id = cursor.lastrowid
            # Confirma a transação
            self.conn.commit()
            # Feedback para o usuário
            print(f"Usuário cadastrado com sucesso! ID: {usuario_id}")
        except sqlite3.IntegrityError:
            # Caso a identificação já exista (é UNIQUE)
            print("Erro: Já existe um usuário com este número de identificação!")
        except sqlite3.Error as e:
            # Outros erros de banco de dados
            print(f"Erro ao cadastrar usuário: {e}")

    def cadastrar_livro(self):
        # Interface para cadastro de novos livros no sistema
        print("\n")
        print("-" * 40)
        print("            Cadastro de Livro     ")
        print("-" * 40)

        # Validação dos campos obrigatórios
        while True:
            titulo = input("Título: ").strip()
            if not titulo:
                print("Erro: O título é obrigatório!")
                continue

            autor = input("Autor: ").strip()
            if not autor:
                print("Erro: O autor é obrigatório!")
                continue

            break  # Campos obrigatórios preenchidos

        # Validação do ano (deve ser um número inteiro)
        while True:
            try:
                ano = int(input("Ano de publicação: "))
                break
            except ValueError:
                print("Por favor, digite um ano válido!")

        # Validação do número de cópias (deve ser número positivo)
        while True:
            try:
                copias = int(input("Número de cópias: "))
                if copias < 0:
                    print("O número de cópias não pode ser negativo!")
                    continue
                break
            except ValueError:
                print("Por favor, digite um número válido!")

        try:
            cursor = self.conn.cursor()
            # Insere o novo livro com todas as cópias disponíveis inicialmente
            cursor.execute('''
            INSERT INTO livros (titulo, autor, ano_publicacao, copias_disponiveis, copias_totais)
            VALUES (?, ?, ?, ?, ?)
            ''', (titulo, autor, ano, copias, copias))

            # Obtém e exibe o ID gerado
            livro_id = cursor.lastrowid
            self.conn.commit()
            print(f"Livro cadastrado com sucesso! ID: {livro_id}")
        except sqlite3.Error as e:
            print(f"Erro ao cadastrar livro: {e}")

    def emprestar_livro(self):
        # Gerencia todo o processo de empréstimo de livros
        print("\n")
        print("-" * 40)
        print("            Empréstimo de Livro     ")
        print("-" * 40)

        # Verificar usuário
        identificacao = input("Digite o número de identificação do usuário: ")
        cursor = self.conn.cursor()
        # Busca o usuário pelo número de identificação
        cursor.execute('SELECT id FROM usuarios WHERE identificacao = ?', (identificacao,))
        usuario = cursor.fetchone()

        if not usuario:
            print("Usuário não encontrado!")
            return # Sai da função se usuário não existe

        usuario_id = usuario[0] # Pega o ID do usuário

        # Buscar livro
        termo = input("Digite o título ou autor do livro: ")
        # Busca livros que tenham o termo no título ou no autor
        cursor.execute('''
        SELECT id, titulo, autor, copias_disponiveis
        FROM livros
        WHERE titulo LIKE ? OR autor LIKE ?
        ''', (f'%{termo}%', f'%{termo}%'))

        livros = cursor.fetchall()

        if not livros:
            print("Nenhum livro encontrado com esse termo!")
            return

        # Mostra os livros encontrados
        print("\nLivros encontrados:")
        for i, livro in enumerate(livros, 1):
            print(f"{i} - ID: {livro[0]} - {livro[1]} por {livro[2]} - {livro[3]} cópias disponíveis")
            print("\nNão há cópias disponíveis no momento")

        # Selecionar livro para empréstimo
        try:
            escolha = int(input("\nEscolha o número do livro: ")) - 1
            if escolha < 0 or escolha >= len(livros):
                print("Escolha inválida!")
                return
        except ValueError:
            print("Por favor, digite um número válido!")
            return

        livro_id, titulo, autor, copias = livros[escolha]

        # Verifica se há cópias disponíveis
        if copias <= 0:
            print("Desculpe, não há cópias disponíveis deste livro!")
            return

        # Registar empréstimo
        data_emprestimo = datetime.now().strftime('%Y-%m-%d')
        data_devolucao = (datetime.now() + timedelta(days=14)).strftime('%Y-%m-%d')

        try:
            # Insere registro na tabela de empréstimos
            cursor.execute('''
            INSERT INTO emprestimos
            (livro_id, usuario_id, data_emprestimo, data_devolucao_prevista)
            VALUES (?, ?, ?, ?)
            ''', (livro_id, usuario_id, data_emprestimo, data_devolucao))

            # Obter o ID do empréstimo
            emprestimo_id = cursor.lastrowid

            # Atualizar o estoque (diminui uma cópia disponível)
            cursor.execute('''
            UPDATE livros
            SET copias_disponiveis = copias_disponiveis - 1
            WHERE id = ?
            ''', (livro_id,))

            self.conn.commit() # Confirma as alterações
            print(f"Empréstimo realizado com sucesso! ID: {emprestimo_id}")
            print(f"Devolução até {data_devolucao}")
        except sqlite3.Error as e:
            print(f"Erro ao realizar empréstimo: {e}")

    def devolver_livro(self):
        print("\n")
        print("-" * 40)
        print("            Devolução de Livro     ")
        print("-" * 40)

        # Buscar usuário
        identificacao = input("Digite o número de identificação do usuário: ")
        cursor = self.conn.cursor()
        # Busca usuário pelo número de identificação
        cursor.execute('SELECT id, nome FROM usuarios WHERE identificacao = ?', (identificacao,))
        usuario = cursor.fetchone()

        if not usuario:
            print("Usuário não encontrado!")
            return

        usuario_id, usuario_nome = usuario # Desempacota os dados

        # Buscar empréstimos ativos do usuário
        cursor.execute('''
        SELECT e.id, l.titulo, e.data_emprestimo, e.data_devolucao_prevista
        FROM emprestimos e
        JOIN livros l ON e.livro_id = l.id
        WHERE e.usuario_id = ? AND e.data_devolucao_real IS NULL
        ORDER BY e.data_devolucao_prevista
        ''', (usuario_id,))

        emprestimos = cursor.fetchall()

        if not emprestimos:
            print("Nenhum empréstimo ativo para este usuário!")
            return

        # Mostrar os empréstimos ativos
        print("\nEmpréstimos ativos:")
        for i, (emp_id, titulo, data_emp, data_dev) in enumerate(emprestimos, 1):
            print(f"{i}. {titulo} - Emprestado em: {data_emp} (Devolver até: {data_dev})")

        # Selecionar empréstimo para devolver
        try:
            escolha = int(input("\nDigite o número do empréstimo para devolver: ")) - 1
            if escolha < 0 or escolha >= len(emprestimos):
                print("Opção inválida!")
                return

            emp_id, titulo, _, _ = emprestimos[escolha] # Pega dados do empréstimo

            # Registrar devolução
            data_devolucao = datetime.now().strftime('%Y-%m-%d')
            # Atualiza a data de devolução real
            cursor.execute('UPDATE emprestimos SET data_devolucao_real = ? WHERE id = ?',
                         (data_devolucao, emp_id))

            # Atualizar estoque (aumenta uma cópia disponível)
            cursor.execute('''
            UPDATE livros SET copias_disponiveis = copias_disponiveis + 1
            WHERE id = (SELECT livro_id FROM emprestimos WHERE id = ?)
            ''', (emp_id,))

            self.conn.commit() # Confirma as alterações
            print(f"\nLivro '{titulo}' devolvido com sucesso {data_devolucao}!")

        except ValueError:
            print("Por favor, digite um número válido!")
        except sqlite3.Error as e:
            print(f"Erro ao registrar devolução: {e}")
            self.conn.rollback() # Desfaz alterações em caso de erro

    def consultar_livros(self):
        # Oferece diferentes formas de consultar o acervo de livros
        # Interface menu de consulta
        print("\n")
        print("-" * 40)
        print("           Consulta de Livros    ")
        print("-" * 40)
        print("1 | Por título")
        print("2 | Por autor")
        print("3 | Por ano de publicação")
        print("4 | Todos os livros")

        try:
            # Recebe a opção do usuário
            opcao = int(input("Escolha uma opção de busca: "))
        except ValueError:
            print("Opção inválida!")
            return

        cursor = self.conn.cursor()

        # Busca por título
        if opcao == 1:
            termo = input("Digite o título ou parte dele: ")
            # Busca livros onde o título contém o termo (case insensitive)
            cursor.execute('''
            SELECT id, titulo, autor, ano_publicacao, copias_disponiveis
            FROM livros
            WHERE titulo LIKE ?
            ORDER BY titulo
            ''', (f'%{termo}%',))

        # Busca por autor
        elif opcao == 2:
            termo = input("Digite o nome do autor ou parte dele: ")
            # Busca livros onde o autor contém o termo
            cursor.execute('''
            SELECT id, titulo, autor, ano_publicacao, copias_disponiveis
            FROM livros
            WHERE autor LIKE ?
            ORDER BY autor, titulo
            ''', (f'%{termo}%',))

        # Busca por ano de publicação
        elif opcao == 3:
            try:
                ano = int(input("Digite o ano de publicação: "))
                # Busca livros publicados no ano específico
                cursor.execute('''
                SELECT id, titulo, autor, ano_publicacao, copias_disponiveis
                FROM livros
                WHERE ano_publicacao = ?
                ORDER BY titulo
                ''', (ano,))
            except ValueError:
                print("Ano inválido!")
                return

        # Lista todos os livros
        elif opcao == 4:
            cursor.execute('''
            SELECT id, titulo, autor, ano_publicacao, copias_disponiveis
            FROM livros
            ORDER BY titulo
            ''')
        else:
            print("Opção inválida!")
            return

        # Obtém e processa os resultados
        livros = cursor.fetchall()

        if not livros:
            print("Nenhum livro encontrado com os critérios especificados!")
            return

        # Exibe os resultados em formato de tabela
        print("\nResultados da busca:")
        print("-" * 90)
        # Cabeçalho da tabela
        print(f"{'ID':<5} | {'Título':<30} | {'Autor':<20} | {'Ano':<6} | {'Disponíveis':<10}")
        print("-" * 90)
        # Linhas com os dados dos livros
        for livro in livros:
            print(f"{livro[0]:<5} | {livro[1]:<30} | {livro[2]:<20} | {livro[3]:<6} | {livro[4]:<10}")

    def gerar_relatorios(self):
        # Gera diversos tipos de relatórios gerenciais
        # Interface do menu de relatórios
        print("\n")
        print("-" * 40)
        print("              Relatórios    ")
        print("-" * 40)
        print("1 | Livros disponíveis")
        print("2 | Livros emprestados")
        print("3 | Usuários cadastrados")
        print("4 | Empréstimos ativos")
        print("5 | Histórico de empréstimos")
        print("-" * 40)

        try:
            opcao = int(input("Escolha uma opção de relatório: "))
        except ValueError:
            print("Opção inválida!")
            return

        cursor = self.conn.cursor()

        # Gera um ID único baseado na data/hora atual
        relatorio_id = datetime.now().strftime("%Y%m%d%H%M%S")
        print(f"\n=== Relatório ID: {relatorio_id} ===")
        print("-" * 50)

        # Livros disponíveis
        if opcao == 1:
            cursor.execute('''
            SELECT id, titulo, autor, copias_disponiveis, copias_totais
            FROM livros
            WHERE copias_disponiveis > 0
            ORDER BY titulo
            ''')
            livros = cursor.fetchall()

            if not livros:
                print("Nenhum livro disponível no momento!")
                return

            print("\nLivros disponíveis:")
            print("-" * 90)
            print(f"{'ID':<5} | {'Título':<30} | {'Autor':<20} | {'Disponíveis':<12} | {'Total':<12}")
            print("-" * 90)
            for livro in livros:
                print(f"{livro[0]:<5} | {livro[1]:<30} | {livro[2]:<20} | {livro[3]:<12} | {livro[4]:<12}")

        # Livros emprestados
        elif opcao == 2:
            cursor.execute('''
            SELECT l.id, l.titulo, l.autor,
                   COUNT(e.id) as qtd_emprestimos,
                   l.copias_totais - l.copias_disponiveis as total_emprestados
            FROM livros l
            JOIN emprestimos e ON l.id = e.livro_id
            WHERE e.data_devolucao_real IS NULL
            GROUP BY l.id
            ORDER BY qtd_emprestimos DESC
            ''')
            livros = cursor.fetchall()

            if not livros:
                print("Nenhum livro emprestado no momento!")
                return

            print("\nLivros emprestados:")
            print("-" * 90)
            print(f"{'ID':<5} | {'Título':<30} | {'Autor':<20} | {'Empréstimos ativos':<18} | {'Total emprestado':<15}")
            print("-" * 90)
            for livro in livros:
                print(f"{livro[0]:<5} | {livro[1]:<30} | {livro[2]:<20} | {livro[3]:<18} | {livro[4]:<15}")

        # Usuários cadastrados
        elif opcao == 3:
            cursor.execute('''
            SELECT id, nome, identificacao, contato
            FROM usuarios
            ORDER BY nome
            ''')
            usuarios = cursor.fetchall()

            if not usuarios:
                print("Nenhum usuário cadastrado!")
                return

            print("\nUsuários cadastrados:")
            print("-" * 90)
            print(f"{'ID':<5} | {'Nome':<30} | {'Identificação':<15} | {'Contato':<30}")
            print("-" * 90)
            for usuario in usuarios:
                print(f"{usuario[0]:<5} | {usuario[1]:<30} | {usuario[2]:<15} | {usuario[3]:<30}")

        # Empréstimos ativos
        elif opcao == 4:
            cursor.execute('''
            SELECT e.id, u.nome, l.titulo, e.data_emprestimo, e.data_devolucao_prevista
            FROM emprestimos e
            JOIN usuarios u ON e.usuario_id = u.id
            JOIN livros l ON e.livro_id = l.id
            WHERE e.data_devolucao_real IS NULL
            ORDER BY e.data_devolucao_prevista
            ''')
            emprestimos = cursor.fetchall()

            if not emprestimos:
                print("Nenhum empréstimo ativo no momento!")
                return

            print("\nEmpréstimos ativos:")
            print("-" * 110)
            print(f"{'ID':<5} | {'Usuário':<25} | {'Livro':<30} | {'Data empréstimo':<15} | {'Devolução prevista':<15}")
            print("-" * 110)
            for emp in emprestimos:
                print(f"{emp[0]:<5} | {emp[1]:<25} | {emp[2]:<30} | {emp[3]:<15} | {emp[4]:<15}")

        # Histórico de empréstimos
        elif opcao == 5:
            cursor.execute('''
            SELECT e.id, u.nome, l.titulo, e.data_emprestimo, e.data_devolucao_prevista,
                   e.data_devolucao_real,
                   CASE
                       WHEN e.data_devolucao_real IS NULL THEN 'Em atraso'
                       WHEN e.data_devolucao_real > e.data_devolucao_prevista THEN 'Devolvido com atraso'
                       ELSE 'Devolvido no prazo'
                   END as status
            FROM emprestimos e
            JOIN usuarios u ON e.usuario_id = u.id
            JOIN livros l ON e.livro_id = l.id
            ORDER BY e.data_emprestimo DESC
            LIMIT 50
            ''')
            historico = cursor.fetchall()

            if not historico:
                print("Nenhum registro de empréstimo encontrado!")
                return

            print("\nHistórico de empréstimos (últimos 50):")
            print("-" * 130)
            print(f"{'ID':<5} | {'Usuário':<20} | {'Livro':<25} | {'Empréstimo':<12} | {'Prevista':<12} | {'Real':<12} | {'Status':<20}")
            print("-" * 130)
            for item in historico:
                devolucao_real = item[5] if item[5] else "---"
                print(f"{item[0]:<5} | {item[1]:<20} | {item[2]:<25} | {item[3]:<12} | {item[4]:<12} | {devolucao_real:<12} | {item[6]:<20}")

        else:
            print("Opção inválida!")

    def menu_principal(self):
        # Loop infinito para o menu
        while True:
            print("-" * 50)
            print("      Sistema de Gerenciamento de Biblioteca     ")
            print("-" * 50)
            print(" 1  |  Cadastrar Livro")
            print(" 2  |  Cadastrar Usuário")
            print(" 3  |  Emprestar Livro")
            print(" 4  |  Devolver Livro")
            print(" 5  |  Consultar Livros")
            print(" 6  |  Gerar Relatórios")
            print(" 0  |  Sair")
            print("-" * 50)

            try:
                opcao = int(input("Escolha uma opção: "))
            except ValueError:
                print("Por favor, digite um número válido!")
                continue # Volta ao início do loop

             # Chama a função correspondente à opção selecionada
            if opcao == 1:
                self.cadastrar_livro()
            elif opcao == 2:
                self.cadastrar_usuario()
            elif opcao == 3:
                self.emprestar_livro()
            elif opcao == 4:
                self.devolver_livro()
            elif opcao == 5:
                self.consultar_livros()
            elif opcao == 6:
                self.gerar_relatorios()
            elif opcao == 0:
                print("Saindo do sistema...")
                self.conn.close()# Fecha a conexão com o banco de dados
                break  # Sai do lop while
            else:
                print("Opção inválida!")

# Iniciar o sistema
if __name__ == "__main__":
    print("    *** Bem-vindo ao Sistema de Biblioteca! ***")
    biblioteca = Biblioteca() # Cria uma instância da classe Biblioteca
    biblioteca.menu_principal()  # Inicia o menu principal

    *** Bem-vindo ao Sistema de Biblioteca! ***
--------------------------------------------------
      Sistema de Gerenciamento de Biblioteca     
--------------------------------------------------
 1  |  Cadastrar Livro
 2  |  Cadastrar Usuário
 3  |  Emprestar Livro
 4  |  Devolver Livro
 5  |  Consultar Livros
 6  |  Gerar Relatórios
 0  |  Sair
--------------------------------------------------
